In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ParameterGrid
%matplotlib inline
import random
import time

In [2]:
import motif_processor
reload(motif_processor)

<module 'motif_processor' from 'motif_processor.pyc'>

In [3]:
MotifProcObj = motif_processor.MotifProcessor(ENCODE_only=True, 
                                                   cell_line="HepG2", 
                                                  meme_file="/home/alvin/Dropbox/Lab/CNN/data/meme/HepG2_pos_steme/steme.txt")
motif_tensor, motif_names = MotifProcObj.generate_custom_CNN_filters(max_length = 18, padding = 0, truncate = False)


In [4]:
import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu1')

Using gpu device 1: TITAN X (Pascal) (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/home/alvin/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [5]:
HepG2MotifProcObj = motif_processor.MotifProcessor(ENCODE_only=False, 
                                                   cell_line="HepG2", 
                                                   meme_file="/home/alvin/Dropbox/Lab/CNN/data/meme/HepG2_pos_steme/steme.txt")
HepG2_motif_tensor, HepG2_motif_names = HepG2MotifProcObj.generate_custom_CNN_filters(max_length = 18, padding = 0, truncate = False)

K562MotifProcObj = motif_processor.MotifProcessor(ENCODE_only=False, 
                                                   cell_line="K562", 
                                                   meme_file="/home/alvin/Dropbox/Lab/CNN/data/meme/K562_pos_steme/steme.txt")
K562_motif_tensor, K562_motif_names = K562MotifProcObj.generate_custom_CNN_filters(max_length = 18, padding = 0, truncate = False)


LCLMotifProcObj = motif_processor.MotifProcessor(ENCODE_only=False, 
                                                   cell_line="LCL", 
                                                   meme_file="/home/alvin/Dropbox/Lab/CNN/data/meme/LCL_pos_steme/steme.txt")
LCL_motif_tensor, LCL_motif_names = LCLMotifProcObj.generate_custom_CNN_filters(max_length = 18, padding = 0, truncate = False)
MotifProcObj = motif_processor.MotifProcessor(ENCODE_only=True, 
                                                   cell_line="HepG2", 
                                                  meme_file="/home/alvin/Dropbox/Lab/CNN/data/meme/HepG2_pos_steme/steme.txt")
motif_tensor, motif_names = MotifProcObj.generate_custom_CNN_filters(max_length = 18, padding = 0, truncate = False)


In [6]:
motifDistanceProcessorObj = motif_processor.MotifDistanceProcessor([],[],[])
binary_mat = motifDistanceProcessorObj.process_Pouya(motif_names)

In [7]:
import common_utils
reload(common_utils)

/home/alvin/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/home/alvin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/alvin/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


<module 'common_utils' from 'common_utils.pyc'>

In [8]:
CrossValProcObj = common_utils.CrossValProcessor(output_dirs = ["./HEPG2_act_1_31/", "./HEPG2_rep_1_31/", 
                                                                      "./K562_act_1_31/", "./K562_act_1_31/",
                                                                      "./LCL_act_alt_1_31/"])

HepG2_binObj = CrossValProcObj.HepG2_binObj
K562_binObj = CrossValProcObj.K562_binObj
LCL_binObj = CrossValProcObj.LCL_binObj
binObjs = [HepG2_binObj, K562_binObj, LCL_binObj]
CvDicts = [CrossValProcObj.HepG2_dict, CrossValProcObj.K562_dict, CrossValProcObj.LCL_dict]
motif_tensors = [motif_tensor, motif_tensor, motif_tensor]
motif_names = [motif_names, motif_names, motif_names]
names=["HepG2", "K562", "LCL"]
dirs = ["./HepG2_2_16/", "./K562_2_16/", "./LCL_2_16/"]


In [9]:
import cv_engine 
reload(cv_engine)
HepG2DataConvObj = cv_engine.DataConvolver(HepG2_binObj.merged_tensor, HepG2_binObj.merged_RC_tensor, HepG2_binObj.merged_tar, motif_tensor.shape, features_mat=HepG2_binObj.features_mat, should_conv=False)
K562DataConvObj = cv_engine.DataConvolver(K562_binObj.merged_tensor, K562_binObj.merged_RC_tensor, K562_binObj.merged_tar, motif_tensor.shape, features_mat=K562_binObj.features_mat,should_conv=False) 
LCLDataConvObj = cv_engine.DataConvolver(LCL_binObj.merged_tensor, LCL_binObj.merged_RC_tensor, LCL_binObj.merged_tar, motif_tensor.shape,features_mat=LCL_binObj.features_mat, should_conv=False) 

Using Theano backend.


In [13]:
import comVar_keras
reload(comVar_keras)

<module 'comVar_keras' from 'comVar_keras.pyc'>

In [14]:
binObjs = [HepG2DataConvObj,K562DataConvObj,LCLDataConvObj]
names=['HepG2','K562','LCL']
model_names = ['Meta', 'Double', 'Simple']

In [15]:
def process(binObj, val_binObj, name):
    start = time.time()
    valamount = 5000
    MetaModel = comVar_keras.MetaKerasModel(binObj, motif_tensor, binary_mat)
    DoubleKerasModel = comVar_keras.DoubleKerasModel(binObj, motif_tensor)
    SimpleModel = comVar_keras.SimpleKerasModel(binObj)
    Models = [MetaModel, DoubleKerasModel, SimpleModel]
    model_names = ['Meta', 'Double', 'Simple']
    pool_width = 4
    weighted = 1
    L2 = 0.1
    L1 = 0
    dropout = 0.1
    val_set = np.arange(val_binObj.X.shape[0])
    np.random.shuffle(val_set)
    val_range = 5000
    val_indices = val_set[:val_range]
    print(val_indices[:5])
    for model_idx, Model in enumerate(Models):
        Model.build_model(dropout=dropout, L1=L1, L2=L2, pool_width=pool_width, weighted=weighted)
        pos_weight, neg_weight = cv_engine.get_weights(binObj.Y)
        train_history = Model.train(binObj.X, binObj.X_RC, binObj.Y, 
                                    neg_weight, pos_weight, 
                                    outside_eval = ([val_binObj.X[val_indices], val_binObj.X_RC[val_indices]], val_binObj.Y[val_indices])) 
        comVar_keras.dump_model(Model.model, name = "./models/%s_%s"%(name, model_names[model_idx]))
        print "Done building %s for %s in %0.05f seconds"%(model_names[model_idx], name, time.time()-start)

In [16]:
process(binObjs[1], binObjs[2], 'K562')

[ 7117  3769 18954 19405 13915]
Done dumping ./models/K562_Meta
Done building Meta for K562 in 623.21949 seconds
Done dumping ./models/K562_Double
Done building Double for K562 in 822.18272 seconds
Done dumping ./models/K562_Simple
Done building Simple for K562 in 871.22748 seconds


In [17]:
process(binObjs[0], binObjs[1], 'HepG2')


[11738  4011 12293  3441  8687]
Done dumping ./models/HepG2_Meta
Done building Meta for HepG2 in 590.52758 seconds
Done dumping ./models/HepG2_Double
Done building Double for HepG2 in 773.14601 seconds
Done dumping ./models/HepG2_Simple
Done building Simple for HepG2 in 794.91568 seconds


In [18]:
process(binObjs[2], binObjs[1], 'LCL')


[ 6214 11693  9784  3717  7330]
Done dumping ./models/LCL_Meta
Done building Meta for LCL in 924.80980 seconds
Done dumping ./models/LCL_Double
Done building Double for LCL in 1224.19781 seconds
Done dumping ./models/LCL_Simple
Done building Simple for LCL in 1258.68504 seconds
